在大型分类变量的情况下 使用one-hot编码似乎会使得矩阵过于稀疏 内存利用率也较低

一般处理方式有两种：

1. 特征散列化 (适用于线性模型)
2. 分箱计数 (适用于线性模型和树模型)

# 散列特征化
散列特征化本质就是哈希+取余的方法

散列特征化通过哈希+取余的方法 将所有的数据映射到[1,m]的区间内 m个分箱中

散列特征化节约了内存空间 但是失去了特征的可解释性 只变成了初始特征的某种聚合

In [ ]:
# 散列特征化实践

In [1]:
import pandas as pd
import json
from sklearn.feature_extraction import FeatureHasher

In [2]:
dataset_root_path = 'feature_engineering/dataset/'

In [3]:
f = open(dataset_root_path + 'yelp_dataset/yelp_academic_dataset_review.json')

In [4]:
js = []
for i in range(10000):
    js.append(json.loads(f.readline()))
f.close()

In [5]:
review_df = pd.DataFrame(js)

In [6]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [7]:
m = len(review_df['business_id'].unique())
m

4299

In [8]:
h = FeatureHasher(n_features=m, input_type='string')
FH = h.transform(review_df['business_id'])

In [10]:
FH

<10000x4299 sparse matrix of type '<class 'numpy.float64'>'
	with 186129 stored elements in Compressed Sparse Row format>

In [12]:
FH.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
FH.toarray()[0].sum()

6.0

In [19]:
FH.toarray()[5].sum()

-2.0

In [21]:
from sys import getsizeof

In [22]:
print('Our pandas Series, in bytes: ', getsizeof(review_df['business_id']))
print('Our hashed numpy array, in bytes: ', getsizeof(f))

Our pandas Series, in bytes:  790144
Our hashed numpy array, in bytes:  208


## 分箱计数

分箱计数的思想：分箱计数不使用分类变量的值作为特征，而是使用目标变量取这个值的条件概率

换句话说 不对分类变量的值进行编码 而是要计算分类变量值与要预测的目标变量之间的相关关系

分享计数会将二值列编码为一个单独的特征 是0-1的实数值

这个分箱计数的统计量可以是优势比 对数优势比 也可以是别的东西

简而言之 分箱计数将一个分类变量转化为与其值相关的统计量

其将一个大型的 稀疏的 二值的分类变量(one-hot等) 转换为小巧的 密集的 实数型的数值表示

In [31]:
df = pd.read_csv(dataset_root_path + 'Avazu/train.csv', nrows=10000)
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [32]:
len(df['device_id'].unique())

1075

In [ ]:
# 对于每个类别 需要计算[counts, p(click), p(no click), p(click)/p(no click)]

In [41]:
def click_counting(x, bin_column):
    # 首先过滤出有点击的 click>0 之后用value_counts取计数
    clicks = pd.Series(x[x['click'] > 0][bin_column].value_counts(),name='clicks')
    # 过滤出无点击的 click=0
    no_clicks = pd.Series(x[x['click'] < 1][bin_column].value_counts(),name='no_clicks')
    counts = pd.DataFrame([clicks,no_clicks]).T.fillna('0')
    # 将两个表拼接起来
    counts['total_clicks'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64')
    # 所有点击是 click + no_click
    return counts

In [34]:
def bin_counting(counts):
    counts['N+'] =counts['clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['N-'] =counts['no_clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['log_N+'] = counts['N+'].divide(counts['N-'])
    bin_counts = counts.filter(items = ['N+','N-','log_N+'])
    return counts, bin_counts

In [36]:
df.filter(items= ['device_id', 'click'])

,device_id,click
0,a99f214a,0
1,a99f214a,0
2,a99f214a,0
3,a99f214a,0
4,a99f214a,0
...,...,...
9995,a99f214a,0
9996,a99f214a,1
9997,a99f214a,1
9998,a99f214a,0


In [39]:
df['device_id'].value_counts()

a99f214a    8724
c357dbff      17
a167aa83       9
3c0208dc       9
31da1bd0       8
            ... 
ce71b050       1
c4f13677       1
e0f70006       1
18193fc4       1
24490d64       1
Name: device_id, Length: 1075, dtype: int64

In [42]:
bin_column = 'device_id'
device_clicks = click_counting(df.filter(items= [bin_column, 'click']),bin_column)

In [44]:
device_all, device_bin_counts = bin_counting(device_clicks)

In [45]:
device_all

,clicks,no_clicks,total_clicks,N+,N-,log_N+
a99f214a,1561.0,7163.0,8724,0.178932,0.821068,0.217925
25635c83,3.0,0,3,1.000000,0.000000,inf
9af87478,2.0,0,2,1.000000,0.000000,inf
c357dbff,2.0,15.0,17,0.117647,0.882353,0.133333
e62f1261,2.0,3.0,5,0.400000,0.600000,0.666667
...,...,...,...,...,...,...
466a360a,0,1.0,1,0.000000,1.000000,0.000000
ed76a0e7,0,1.0,1,0.000000,1.000000,0.000000
9fd18705,0,1.0,1,0.000000,1.000000,0.000000
1fc5413e,0,1.0,1,0.000000,1.000000,0.000000


In [46]:
device_bin_counts

,N+,N-,log_N+
a99f214a,0.178932,0.821068,0.217925
25635c83,1.000000,0.000000,inf
9af87478,1.000000,0.000000,inf
c357dbff,0.117647,0.882353,0.133333
e62f1261,0.400000,0.600000,0.666667
...,...,...,...
466a360a,0.000000,1.000000,0.000000
ed76a0e7,0.000000,1.000000,0.000000
9fd18705,0.000000,1.000000,0.000000
1fc5413e,0.000000,1.000000,0.000000


In [47]:
device_all.sort_values(by = 'total_clicks', ascending=False).head(4)

,clicks,no_clicks,total_clicks,N+,N-,log_N+
a99f214a,1561.0,7163.0,8724,0.178932,0.821068,0.217925
c357dbff,2.0,15.0,17,0.117647,0.882353,0.133333
a167aa83,0,9.0,9,0.000000,1.000000,0.000000
3c0208dc,0,9.0,9,0.000000,1.000000,0.000000


## 分享技术细节

### 处理稀有类
和罕见词一样 稀有类也要处理 比如有些用户一年只登陆了一次 如果为了他们多加一些行 会导致数据的浪费

使用back-off方法解决 将稀有类的计数累加到特殊分箱中

即出现数量大于某阈值的不变 而小于某阈值的归类为稀有类 统一处理为一行

### 防止数据泄露
由于分箱计数需要依赖历史数据 而当数据集更新时 就需要重新计算

如果使用同样地数据集计算统计量和训练模型 会造成数据泄露

因为训练信息中在计算统计量的时候 包含了将要预测的数据 造成了数据泄露

数据泄露有多种可能 如测试数据泄漏到训练数据中 未来数据泄漏到过去数据中

一般是使用过去的数据点进行计数-使用当前数据点进行训练-用未来数据点进行测试

可以防止数据泄露

### 无界计数
由于历史数据不断增加 统计量计算中的总数就会不断增加 似乎没有边界

当计数增加非常快的时候 就会造成问题

可以使用归一化或者取log来延缓这种趋势

# 总结
## Plain one-hot encoding
空间使用：O(n) 时间复杂度：O(nk)

### 优点:

容易实现

更高的精度

在线学习特别容易扩展

### 缺点
计算不足

如果类别增加则不能够使用

对线性模型以外的任何其他方法都不可行

对于大数据集需要分布式训练

## Feature hashing
空间使用：O(n) 时间复杂度：O(nm)

### 优点:

容易实现

容易训练

容易扩展到新类别

容易处理稀有类别

在线学习容易扩展
### 缺点

只能够使用线性或核模型

哈希编码很难解释

精度有争议

## Bin-counting
空间使用：O(n+k) 时间复杂度：O(n)

### 优点:
训练快

能够使用树模型

容易扩展到新列类别

容易处理稀有类别

可解释

### 缺点
需要利用历史信息

对于在线学习有困难

会有数据泄露